In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-23 12:02:29 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-23 12:02:29 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-23 12:02:29 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-23 12:02:31,867] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-23 12:02:31 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmp1rgkrkol/test.c -o /tmp/tmp1rgkrkol/test.o
2025-07-23 12:02:31 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmp1rgkrkol/test.o -laio -o /tmp/tmp1rgkrkol/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-23 12:02:32 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpwnsi6xit/test.c -o /tmp/tmpwnsi6xit/test.o
2025-07-23 12:02:32 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpwnsi6xit/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpwnsi6xit/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-23 12:02:33 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-23 12:02:33 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-23 12:02:33 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-23 12:02:33 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-23 12:02:34 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-23 12:02:34 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-07-23 12:02:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-23 12:02:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-23 12:03:22 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-23 12:03:23 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>10<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [7]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-23 12:03:37 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-23 12:03:37 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-23 12:03:37 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-23 12:03:37 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-23 12:03:37 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-23 12:03:37 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-23 12:03:37 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
202

In [8]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-23 12:03:44 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [207]:
import copy
import random
from src.selection.utils import KeyedSet, get_first_token_id
from src.selection.data import SelectionSample
from src.functional import predict_next_token

######################################################################
N_DISTRACTORS = 5
WINDOW_SPEC = {
    mt.layer_name_format: 1,
    mt.mlp_module_name_format: 9,
    mt.attn_module_name_format: 9,
}
module_name_format = mt.layer_name_format
# module_name_format = mt.mlp_module_name_format
# module_name_format = mt.attn_module_name_format

######################################################################

def get_counterfactual_samples_on_pivot_entity():
    patch_category = random.choice(list(people_by_category.keys()))
    patch_subj, patch_obj = random.sample(people_by_category[patch_category].values, 2)
    logger.info(
        f"Patch category: {patch_category}, subject: {patch_subj}, object: {patch_obj}"
    )

    clean_category = random.choice(
        list(set(people_by_category.keys()) - {patch_category})
    )
    clean_subj, clean_obj = random.sample(people_by_category[clean_category].values, 2)
    logger.info(
        f"Clean category: {clean_category}, subject: {clean_subj}, object: {clean_obj}"
    )

    distractors = []
    other_categories = random.sample(
        list(set(people_by_category.keys()) - {patch_category, clean_category}),
        k=N_DISTRACTORS - 1,
    )

    for other_category in other_categories:
        distractors.append(
            random.choice(
                (
                    people_by_category[other_category]
                    - KeyedSet(
                        [patch_obj, clean_obj] + distractors, tokenizer=mt.tokenizer
                    )
                ).values
            )
        )

    patch_options = [patch_obj, clean_obj] + distractors
    random.shuffle(patch_options)
    patch_obj_idx = patch_options.index(patch_obj)
    logger.info(f"{patch_obj_idx=} | {patch_options}")

    clean_options = copy.deepcopy(patch_options)
    while (
        clean_options.index(clean_obj) == patch_obj_idx
        or clean_options.index(patch_obj) == patch_obj_idx
    ):
        random.shuffle(clean_options)

    clean_obj_idx = clean_options.index(clean_obj)

    logger.info(f"{clean_obj_idx=} | {clean_options}")

    patch_sample = SelectionSample(
        subj=patch_subj,
        obj=patch_obj,
        obj_idx=patch_obj_idx,
        obj_token_id=get_first_token_id(patch_obj, mt.tokenizer, prefix=" "),
        options=patch_options,
        metadata={"category": patch_category},
    )
    clean_sample = SelectionSample(
        subj=clean_subj,
        obj=clean_obj,
        obj_idx=clean_obj_idx,
        obj_token_id=get_first_token_id(clean_obj, mt.tokenizer, prefix=" "),
        options=clean_options,
        metadata={
            "category": clean_category,
        },
    )

    for sample in [patch_sample, clean_sample]:
        pred = predict_next_token(
            mt=mt,
            inputs=sample.prompt,
        )[0]
        logger.info(f"{sample.subj} -> {sample.obj} | pred={[str(p) for p in pred]}")
        if pred[0].token_id != sample.obj_token_id:
            logger.error(
                f'Prediction mismatch: {pred[0].token_id}["{mt.tokenizer.decode(pred[0].token_id)}"] != {sample.obj_token_id}["{mt.tokenizer.decode(sample.obj_token_id)}"]'
            )
            return get_counterfactual_samples_on_pivot_entity()
        sample.prediction = pred

    return patch_sample, clean_sample

In [208]:
from src.selection.data import SelectionSample, get_random_sample

patch_sample, clean_sample = get_counterfactual_samples_on_pivot_entity()

print(patch_sample.prompt, " >> ", f'"{patch_sample.obj}"')
print(f"{[str(p) for p in patch_sample.prediction]}")
print(clean_sample.prompt, " >> ", f'"{clean_sample.obj}"')
print(f"{[str(p) for p in clean_sample.prediction]}")

2025-07-23 14:43:56 __main__ INFO     Patch category: politician, subject: Barack Obama, object: Ron DeSantis
2025-07-23 14:43:56 __main__ INFO     Clean category: fashion designer, subject: Michael Kors, object: Demna Gvasalia


2025-07-23 14:43:56 __main__ INFO     patch_obj_idx=5 | ['Demna Gvasalia', 'Danielle Steel', 'Dak Prescott', 'Deontay Wilder', 'James Cameron', 'Ron DeSantis']
2025-07-23 14:43:56 __main__ INFO     clean_obj_idx=1 | ['Danielle Steel', 'Demna Gvasalia', 'James Cameron', 'Deontay Wilder', 'Ron DeSantis', 'Dak Prescott']
2025-07-23 14:43:57 __main__ INFO     Barack Obama -> Ron DeSantis | pred=['" Ron"[14662] (p=0.406, logit=18.625)', '" Barack"[24448] (p=0.170, logit=17.750)', '" None"[2290] (p=0.103, logit=17.250)', '" The"[578] (p=0.055, logit=16.625)', '" James"[7957] (p=0.043, logit=16.375)']
2025-07-23 14:43:57 __main__ INFO     Michael Kors -> Demna Gvasalia | pred=['" Dem"[4829] (p=0.820, logit=20.375)', '" The"[578] (p=0.041, logit=17.375)', '" Michael"[8096] (p=0.028, logit=17.000)', '" Danielle"[72716] (p=0.025, logit=16.875)', '" D"[423] (p=0.017, logit=16.500)']
Which person from the following list has the profession in common with Barack Obama?
Options: Demna Gvasalia, Danie

In [209]:
# patch_sample, clean_sample = clean_sample, patch_sample

In [210]:
from src.tokens import prepare_input
from src.utils.typing import TokenizerOutput

clean_tokenized = prepare_input(
    tokenizer=mt,
    prompts=clean_sample.prompt,
    return_offsets_mapping=True
)

patch_tokenized = prepare_input(
    tokenizer=mt,
    prompts=patch_sample.prompt,
    return_offsets_mapping=True
)

## ATTN module patterns (layerwise average of heads)

In [211]:
from src.attention import get_attention_matrices
from src.functional import interpret_logits
from src.attention import visualize_average_attn_matrix


clean_attn_matrices = get_attention_matrices(
    input=clean_tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=clean_attn_matrices,
    prompt=clean_sample.prompt,
    tokenized=clean_tokenized,
    layer_window=range(35, 36),
)

interpret_logits(tokenizer=mt, logits=clean_attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------


[PredictedToken(token=' Dem', prob=0.83203125, logit=20.5, token_id=4829, metadata=None),
 PredictedToken(token=' The', prob=0.03662109375, logit=17.375, token_id=578, metadata=None),
 PredictedToken(token=' Michael', prob=0.025146484375, logit=17.0, token_id=8096, metadata=None),
 PredictedToken(token=' Danielle', prob=0.025146484375, logit=17.0, token_id=72716, metadata=None),
 PredictedToken(token=' D', prob=0.0152587890625, logit=16.5, token_id=423, metadata=None)]

In [212]:
# visualize_average_attn_matrix(
#     mt=mt,
#     attn_matrices=clean_attn_matrices,
#     prompt=clean_sample.prompt,
#     tokenized=clean_tokenized,
#     layer_window=range(30, 51, 1)
# )

In [213]:
patch_attn_matrices = get_attention_matrices(
    input=patch_tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=patch_attn_matrices,
    prompt=patch_sample.prompt,
    tokenized=patch_tokenized,
    layer_window=range(35, 36),
)

interpret_logits(tokenizer=mt, logits=patch_attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------


[PredictedToken(token=' Ron', prob=0.421875, logit=18.625, token_id=14662, metadata=None),
 PredictedToken(token=' Barack', prob=0.1552734375, logit=17.625, token_id=24448, metadata=None),
 PredictedToken(token=' None', prob=0.1064453125, logit=17.25, token_id=2290, metadata=None),
 PredictedToken(token=' The', prob=0.056884765625, logit=16.625, token_id=578, metadata=None),
 PredictedToken(token=' James', prob=0.04443359375, logit=16.375, token_id=7957, metadata=None)]

In [214]:
# visualize_average_attn_matrix(
#     mt=mt,
#     attn_matrices=patch_attn_matrices,
#     prompt=patch_sample.prompt,
#     tokenized=patch_tokenized,
#     layer_window=range(30, 51, 1)
# )

In [215]:
from src.functional import get_hs, PatchSpec
from src.attention import get_attention_matrices

mt.reset_forward()
input_module = mt.layer_name_format.format(35) + ".input_layernorm"
# input_module = mt.layer_name_format.format(34)

# predictive_tokens = [-3, -2, -1]
predictive_tokens = [-1]

patch_hs = get_hs(
    mt=mt,
    input=patch_tokenized,
    locations=[(input_module, tok_idx) for tok_idx in predictive_tokens],
    return_dict=True
)

int_attn_matrices = get_attention_matrices(
    input=clean_tokenized,
    mt=mt,
    value_weighted=False,
    patches = [PatchSpec(
        location=(input_module, tok_idx),
        patch=patch_hs[(input_module, tok_idx)],
    ) for tok_idx in predictive_tokens]
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=int_attn_matrices,
    prompt=clean_sample.prompt,
    tokenized=clean_tokenized,
    layer_window=range(35, 36),
)

interpret_logits(tokenizer=mt, logits=int_attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------


[PredictedToken(token=' Dem', prob=0.41796875, logit=18.75, token_id=4829, metadata=None),
 PredictedToken(token=' Danielle', prob=0.1748046875, logit=17.875, token_id=72716, metadata=None),
 PredictedToken(token=' None', prob=0.10595703125, logit=17.375, token_id=2290, metadata=None),
 PredictedToken(token=' Michael', prob=0.09326171875, logit=17.25, token_id=8096, metadata=None),
 PredictedToken(token=' The', prob=0.044189453125, logit=16.5, token_id=578, metadata=None)]

### Check if independent processing hypothesis holds
#### Because even after patching in inputs, last layer ATTN still gives an amount of attention to the clean obj tokens. Why?

In [216]:
clean_sample.options

['Danielle Steel',
 'Demna Gvasalia',
 'James Cameron',
 'Deontay Wilder',
 'Ron DeSantis',
 'Dak Prescott']

In [217]:
from src.tokens import find_token_range
from itertools import product

clean_tokenized = prepare_input(
    tokenizer=mt, prompts=clean_sample.prompt, return_offsets_mapping=True
)
clean_offsets = clean_tokenized.pop("offset_mapping")[0]

patches = []
for obj in clean_sample.options:
    obj_range = find_token_range(
        string=clean_sample.prompt,
        substring=obj,
        offset_mapping=clean_offsets,
        tokenizer=mt.tokenizer,
    )

    bare_prompt = f" {obj} is a person."
    bare_tokenized = prepare_input(
        tokenizer=mt, prompts=bare_prompt, return_offsets_mapping=True
    )
    bare_offsets = bare_tokenized.pop("offset_mapping")[0]
    bare_obj_range = find_token_range(
        string=bare_prompt,
        substring=obj,
        offset_mapping=bare_offsets,
        tokenizer=mt.tokenizer,
    )
    print(f"{obj} | {obj_range=} | {bare_obj_range=}")
    assert obj_range[1] - obj_range[0] == bare_obj_range[1] - bare_obj_range[0], (
        f"Object token ranges do not match: {obj_range} vs {bare_obj_range}"
    )

    bare_hs = get_hs(
        mt=mt,
        input=bare_tokenized,
        locations=list(product(mt.layer_names, list(range(*bare_obj_range)))),
        return_dict=True,
    )
    for bare_idx, clean_idx in zip(range(*bare_obj_range), range(*obj_range)):
        patches.extend(
            [
                PatchSpec(
                    location=(module_name, clean_idx),
                    patch=bare_hs[(module_name, bare_idx)],
                )
                for module_name in mt.layer_names
            ]
    )

patches.extend(
    [
        PatchSpec(
            location=(input_module, tok_idx),
            patch=patch_hs[(input_module, tok_idx)],
        )
        for tok_idx in predictive_tokens
    ]
)

obj_replaced_attn_matrices = get_attention_matrices(
    input=clean_tokenized, mt=mt, value_weighted=False, patches=patches
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=obj_replaced_attn_matrices,
    prompt=clean_sample.prompt,
    tokenized=clean_tokenized,
    layer_window=range(35, 36),
)

interpret_logits(tokenizer=mt, logits=obj_replaced_attn_matrices.logits)

Danielle Steel | obj_range=(19, 21) | bare_obj_range=(1, 3)
Demna Gvasalia | obj_range=(22, 27) | bare_obj_range=(1, 6)
James Cameron | obj_range=(28, 30) | bare_obj_range=(1, 3)
Deontay Wilder | obj_range=(31, 36) | bare_obj_range=(1, 6)
Ron DeSantis | obj_range=(37, 41) | bare_obj_range=(1, 5)
Dak Prescott | obj_range=(42, 44) | bare_obj_range=(1, 3)
layer=35


--------------------------------------------------------------------------------


[PredictedToken(token=' Dem', prob=0.435546875, logit=18.125, token_id=4829, metadata=None),
 PredictedToken(token=' None', prob=0.16015625, logit=17.125, token_id=2290, metadata=None),
 PredictedToken(token=' Michael', prob=0.10986328125, logit=16.75, token_id=8096, metadata=None),
 PredictedToken(token=' Ron', prob=0.058837890625, logit=16.125, token_id=14662, metadata=None),
 PredictedToken(token=' The', prob=0.058837890625, logit=16.125, token_id=578, metadata=None)]

## Apply (targetted) Patchscope on different layers

In [149]:
tokenized = prepare_input(
    tokenizer=mt,
    prompts=sample.prompt,
    return_offsets_mapping=True,
)
offset_mapping = tokenized.pop("offset_mapping")[0]


NameError: name 'sample' is not defined

In [14]:
from src.tokens import find_token_range

subj_range = find_token_range(
    string=sample.prompt,
    substring=sample.subj,
    offset_mapping=offset_mapping,
    tokenizer=mt,
    occurrence=-1,
)

print(
    f'{subj_range=} | "{mt.tokenizer.decode(tokenized.input_ids[0][range(*subj_range)])}"'
)

option_ranges = []

for option in sample.options:
    option_range = find_token_range(
        string=sample.prompt,
        substring=option,
        offset_mapping=offset_mapping,
        tokenizer=mt,
        occurrence=-1,
    )
    print(
        f'{option_range=} | "{mt.tokenizer.decode(tokenized.input_ids[0][range(*option_range)])}"'
    )
    option_ranges.append(option_range)

subj_range=(13, 15) | " Trevor Noah"
option_range=(18, 20) | " Zion Williamson"
option_range=(21, 24) | " John Mulaney"
option_range=(25, 28) | " Ariana Grande"
option_range=(29, 31) | " Joe Scarborough"
option_range=(32, 36) | " Andrey Rublev"
option_range=(37, 41) | " Samuel L. Jackson"


In [15]:
from src.functional import patchscope, get_hs
from itertools import product

pred_range = (-2, 0)
token_ranges = [pred_range, subj_range] + option_ranges
token_indices = []
for rng in token_ranges:
    token_indices.extend(list(range(rng[0], rng[1])))

layers = mt.layer_names
locations = list(product(layers, token_indices))

hs = get_hs(mt=mt, input=tokenized, locations=locations, patches=[], return_dict=True)


########################################################################
context_prof = """Name of a person -> their profession
Albert Einstein -> physicist
Michael Jordan -> basketball player
Jack Ma -> entrepreneur
placeholder ->"""

patch_layers = [5]
patch_layers = [mt.layer_name_format.format(idx) for idx in patch_layers]
########################################################################

context = context_prof
context_tok = prepare_input(
    prompts=context_prof, tokenizer=mt, return_offsets_mapping=True
)
cxt_offset_mapping = context_tok.pop("offset_mapping")[0]

h_idx = (
    find_token_range(
        string=context,
        substring="placeholder",
        offset_mapping=cxt_offset_mapping,
        tokenizer=mt.tokenizer,
        occurrence=-1,
    )[1]
    - 1
)

for layer in mt.layer_names[::3]:
    print(f"\n{layer=}")
    for cur_rng in token_ranges:
        print(
            f'\n"{mt.tokenizer.decode(tokenized.input_ids[0][range(*cur_rng)])}" | {cur_rng=}'
        )
        for token_idx in range(cur_rng[0], cur_rng[1]):
            token = mt.tokenizer.decode(tokenized.input_ids[0][token_idx])
            ll_pred = patchscope(
                mt=mt,
                h=hs[(layer, token_idx)],
                context=context,
                placeholder="placeholder",
                context_tokenized=context_tok,
                placeholder_idx=h_idx,
                patch_layers=patch_layers,
                k=15,
            )
            ll_fmt = [
                f'"{pred.token}"[p={pred.prob:.2f}, l={pred.logit:.2f}]'
                for pred in ll_pred
            ]
            print(f'{token_idx=} ["{token}"] => {ll_fmt}')

    print("=" * 150)


layer='model.layers.0'

"Ans:" | cur_rng=(-2, 0)
token_idx=-2 ["Ans"] => ['"?\n"[p=0.05, l=11.19]', '" programmer"[p=0.04, l=11.00]', '"??"[p=0.03, l=10.88]', '"???"[p=0.03, l=10.75]', '" software"[p=0.03, l=10.75]', '" unknown"[p=0.03, l=10.62]', '" student"[p=0.02, l=10.44]', '" singer"[p=0.02, l=10.38]', '" entrepreneur"[p=0.02, l=10.38]', '" artist"[p=0.02, l=10.38]', '"?\n\n"[p=0.02, l=10.31]', '"?"[p=0.02, l=10.25]', '" politician"[p=0.01, l=10.00]', '" actor"[p=0.01, l=10.00]', '" ("[p=0.01, l=9.94]']
token_idx=-1 [":"] => ['"?\n"[p=0.22, l=13.75]', '"?\n\n"[p=0.15, l=13.38]', '"??"[p=0.09, l=12.81]', '"?"[p=0.08, l=12.75]', '"???"[p=0.06, l=12.50]', '" \n"[p=0.02, l=11.56]', '" lawyer"[p=0.02, l=11.50]', '" unknown"[p=0.01, l=11.00]', '" artist"[p=0.01, l=10.75]', '" politician"[p=0.01, l=10.62]', '" software"[p=0.01, l=10.56]', '" programmer"[p=0.01, l=10.50]', '" ("[p=0.01, l=10.25]', '" "[p=0.01, l=10.25]', '" \n\n"[p=0.01, l=10.25]']

" Trevor Noah" | cur_rng=(13, 15)
toke

## Individual Heads

In [218]:
from src.attention import visualize_attn_matrix

###############################################################
N_KV_GROUPS = 1  # individual heads
# N_KV_GROUPS = mt.config.num_key_value_heads  # grouped heads
N_HEADS = mt.config.num_attention_heads
LAYER_RANGE = range(mt.n_layer)
###############################################################

group_id_to_head_indices = {
    i: [i * N_KV_GROUPS + j for j in range(N_KV_GROUPS)]
    for i in range(N_HEADS // N_KV_GROUPS)
}
# attn_info = get_attention_matrices(
#     input=tokenized,
#     mt=mt,
#     # value_weighted=True,
#     patches=[],
# )
# attn_info.attention_matrices.shape

In [219]:
attn_info = {
    "clean": clean_attn_matrices,
    "corrupt": patch_attn_matrices,
    "patch": int_attn_matrices,
    "obj_replaced": obj_replaced_attn_matrices,
}

layer_idx = 35

patch_obj_range = find_token_range(
    string=clean_sample.prompt,
    substring=patch_sample.obj,
    offset_mapping=clean_offsets,
    tokenizer=mt.tokenizer,
    occurrence=-1,
)

clean_obj_range = find_token_range(
    string=clean_sample.prompt,
    substring=clean_sample.obj,
    offset_mapping=clean_offsets,
    tokenizer=mt.tokenizer,
    occurrence=-1,
)


logger.debug(
    f'{patch_obj_range=} | "{mt.tokenizer.decode(clean_tokenized.input_ids[0][range(*patch_obj_range)])}"'
)
logger.debug(
    f'{clean_obj_range=} | "{mt.tokenizer.decode(clean_tokenized.input_ids[0][range(*clean_obj_range)])}"'
)

head_scores = []
run_type = "patch"
for head_idx in range(N_HEADS):
    attn_matrix = attn_info[run_type].attention_matrices[layer_idx, head_idx]
    patch_obj_score = attn_matrix[-1][patch_obj_range[0] : patch_obj_range[1]].sum().item()
    clean_obj_score = attn_matrix[-1][clean_obj_range[0] : clean_obj_range[1]].sum().item()
    head_scores.append({
        "head_idx": head_idx,
        "patch_obj_score": patch_obj_score,
        "clean_obj_score": clean_obj_score,
        "diff": patch_obj_score - clean_obj_score,
    })

head_scores = sorted(head_scores, key=lambda x: x["diff"], reverse=True)
head_scores[:10]

2025-07-23 14:44:21 __main__ DEBUG    patch_obj_range=(37, 41) | " Ron DeSantis"
2025-07-23 14:44:21 __main__ DEBUG    clean_obj_range=(22, 27) | " Demna Gvasalia"


[{'head_idx': 19,
  'patch_obj_score': 0.525848388671875,
  'clean_obj_score': 0.023548126220703125,
  'diff': 0.5023002624511719},
 {'head_idx': 7,
  'patch_obj_score': 0.249237060546875,
  'clean_obj_score': 0.07100296020507812,
  'diff': 0.17823410034179688},
 {'head_idx': 23,
  'patch_obj_score': 0.18902969360351562,
  'clean_obj_score': 0.011474609375,
  'diff': 0.17755508422851562},
 {'head_idx': 0,
  'patch_obj_score': 0.196929931640625,
  'clean_obj_score': 0.025594711303710938,
  'diff': 0.17133522033691406},
 {'head_idx': 17,
  'patch_obj_score': 0.1933746337890625,
  'clean_obj_score': 0.026081085205078125,
  'diff': 0.16729354858398438},
 {'head_idx': 16,
  'patch_obj_score': 0.2028350830078125,
  'clean_obj_score': 0.04482269287109375,
  'diff': 0.15801239013671875},
 {'head_idx': 3,
  'patch_obj_score': 0.125518798828125,
  'clean_obj_score': 0.013723373413085938,
  'diff': 0.11179542541503906},
 {'head_idx': 22,
  'patch_obj_score': 0.1422271728515625,
  'clean_obj_score

In [206]:
# for group_id in group_id_to_head_indices:
for group_id in range(16, 23):
    frm = group_id_to_head_indices[group_id][0]
    to = group_id_to_head_indices[group_id][-1]
    print(
        f"Layer {layer_idx}, Group {group_id} (Heads {frm}-{to})"
    )
    for run_name in attn_info:
        print(run_name)
        attn_matrix = (
            torch.Tensor(
                attn_info[run_name].attention_matrices[layer_idx, frm : to + 1]
            )
            .mean(dim=0)
            .squeeze()
        )

        tokenized = patch_tokenized if run_name == "corrupt" else clean_tokenized

        visualize_attn_matrix(
            attn_matrix=attn_matrix,
            tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
            q_index=-1,
            start_from=1,
        )
    
    print("=" * 150)

Layer 35, Group 16 (Heads 16-16)
clean


corrupt


patch


obj_replaced


Layer 35, Group 17 (Heads 17-17)
clean


...

obj_replaced


Layer 35, Group 22 (Heads 22-22)
clean


corrupt


patch


obj_replaced


In [172]:
from src.functional import get_hs
from src.functional import logit_lens, patchscope, get_module_nnsight
########################################################################
context_prof = """Name of a person -> their profession
Albert Einstein -> physicist
Michael Jordan -> basketball player
Jack Ma -> entrepreneur
placeholder ->"""

patch_layers = [5]
patch_layers = [mt.layer_name_format.format(idx) for idx in patch_layers]
########################################################################

layer_idx, head_idx = 35, 50

visualize_attn_matrix(
    attn_matrix=attn_info.attention_matrices[layer_idx, head_idx],
    tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
    q_index=-1,
    start_from=1,
)

# hs = get_hs(
#     mt=mt,
#     input=tokenized,
#     locations=[(layer_name, -1) for layer_name in mt.layer_names],
#     patches=[],
#     return_dict=True,
# )
with mt.trace(tokenized) as trace:
    res_prev = get_module_nnsight(mt, mt.layer_name_format.format(layer_idx-1))
    head_inp = res_prev.output[0].save()

print(f"{head_inp.shape=}")

ll_pred = logit_lens(
    mt = mt,
    h = head_inp[0, -1, :],
)
print(f'll_pred={[str(pred) for pred in ll_pred]}')

patch_pred = patchscope(
    mt=mt,
    h=head_inp[0, -1, :],
    context=context_prof,
    placeholder="placeholder",
    patch_layers=patch_layers,
    k=15,
)
print(f'patch_pred={[str(pred) for pred in patch_pred]}')

head_inp.shape=torch.Size([1, 43, 8192])
ll_pred=['"adel"[15503] (p=0.005, logit=6.906)', '"ais"[2852] (p=0.003, logit=6.625)', '"u"[84] (p=0.003, logit=6.625)', '"uss"[1892] (p=0.003, logit=6.625)', '" none"[7000] (p=0.003, logit=6.594)']
patch_pred=['"?\n"[18072] (p=0.154, logit=12.125)', '"?\n\n"[24688] (p=0.078, logit=11.438)', '"?"[949] (p=0.044, logit=10.875)', '" \n"[720] (p=0.042, logit=10.812)', '" ("[320] (p=0.027, logit=10.375)', '" ->"[1492] (p=0.021, logit=10.125)', '" "[220] (p=0.021, logit=10.125)', '"??"[9602] (p=0.015, logit=9.812)', '" lawyer"[15779] (p=0.013, logit=9.688)', '" unknown"[9987] (p=0.013, logit=9.625)', '" profession"[4913] (p=0.010, logit=9.438)', '"\n\n"[271] (p=0.010, logit=9.375)', '" artist"[10255] (p=0.010, logit=9.375)', '" software"[3241] (p=0.009, logit=9.312)', '" \n\n"[4815] (p=0.008, logit=9.188)']


In [179]:
import baukit
import types
from src.hooking.llama_attention import LlamaAttentionPatcher
from src.functional import predict_next_token

mt.reset_forward()
attn_block = baukit.get_module(
    mt._model,
    mt.attn_module_name_format.format(layer_idx),
)

patched_attn_matrices = {}
patched_head_contributions={}
attn_block.forward = types.MethodType(
    LlamaAttentionPatcher(
        block_name = mt.attn_module_name_format.format(layer_idx),
        save_attn_for=list(range(mt.config.num_attention_heads)),
        store_attn_matrices = patched_attn_matrices,
        store_head_contributions = patched_head_contributions,
    ),
    attn_block
)

patched_pred = predict_next_token(
    mt = mt,
    inputs = tokenized,
)

mt.reset_forward()

patched_pred

2025-07-22 18:55:53 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.35.self_attn
2025-07-22 18:55:53 src.hooking.llama_attention DEBUG    hidden_shape=(1, 43, -1, 128) | input_shape=torch.Size([1, 43]) | torch.Size([1, 43, 8192])
2025-07-22 18:55:54 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 43, 128]) | key_states.size()=torch.Size([1, 8, 43, 128]) | value_states.size()=torch.Size([1, 8, 43, 128])
o_proj_weight_split.size()=torch.Size([8192, 64, 128])
attn_output.size()=torch.Size([1, 43, 64, 128])


[[PredictedToken(token=' Kanye', prob=0.74609375, logit=20.125, token_id=59817, metadata=None),
  PredictedToken(token=' The', prob=0.061279296875, logit=17.625, token_id=578, metadata=None),
  PredictedToken(token=' Nicholas', prob=0.032958984375, logit=17.0, token_id=40796, metadata=None),
  PredictedToken(token=' K', prob=0.029052734375, logit=16.875, token_id=735, metadata=None),
  PredictedToken(token=' Ed', prob=0.017578125, logit=16.375, token_id=3279, metadata=None)]]

In [174]:
from src.functional import logit_lens
print(f"{patched_attn_matrices[head_idx].shape=}")
print(f"{patched_head_contributions[head_idx].shape=}")

visualize_attn_matrix(
    attn_matrix=patched_attn_matrices[head_idx].squeeze(),
    tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
    q_index=-1,
    start_from=1,
)

logit_lens(
    mt = mt,
    h = patched_head_contributions[head_idx][:, -1],
)

patched_attn_matrices[head_idx].shape=torch.Size([1, 43, 43])
patched_head_contributions[head_idx].shape=torch.Size([1, 43, 8192])


[PredictedToken(token=' breat', prob=0.00958251953125, logit=8.75, token_id=28855, metadata=None),
 PredictedToken(token=' continu', prob=0.006561279296875, logit=8.375, token_id=16513, metadata=None),
 PredictedToken(token=' Deng', prob=0.004364013671875, logit=7.96875, token_id=92829, metadata=None),
 PredictedToken(token=' Daily', prob=0.004119873046875, logit=7.90625, token_id=13690, metadata=None),
 PredictedToken(token=' continuation', prob=0.0038604736328125, logit=7.84375, token_id=42271, metadata=None)]

In [175]:
import baukit
attn_module = baukit.get_module(mt._model, mt.attn_module_name_format.format(layer_idx))
print(f"{attn_module.head_dim=} | {attn_module.num_key_value_groups=}")
attn_module

attn_module.head_dim=128 | attn_module.num_key_value_groups=8


LlamaAttention(
  (q_proj): Linear(in_features=8192, out_features=8192, bias=False)
  (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
  (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
  (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
)

In [176]:
from src.hooking.llama_attention import repeat_kv

hidden_shape = (*tokenized.input_ids.shape, -1, attn_module.head_dim)

query_states = attn_module.q_proj(head_inp).view(hidden_shape).transpose(1, 2)
key_states = attn_module.k_proj(head_inp).view(hidden_shape).transpose(1, 2)
value_states = attn_module.v_proj(head_inp).view(hidden_shape).transpose(1, 2)
print(f"{query_states.shape=}, {key_states.shape=}, {value_states.shape=}")

query_states.shape=torch.Size([1, 64, 43, 128]), key_states.shape=torch.Size([1, 8, 43, 128]), value_states.shape=torch.Size([1, 8, 43, 128])


In [186]:
head_query_out = query_states[:, head_idx, :, :]
head_query_out.shape

torch.Size([1, 43, 128])

In [219]:
q_proj_weight_split = attn_module.q_proj.weight.view(
    N_HEADS, attn_module.head_dim, attn_module.q_proj.in_features
)

print(f"{q_proj_weight_split.shape=}")
head_q_proj_weight = q_proj_weight_split[head_idx]
print(f"{head_q_proj_weight.shape=}")

head_query_out_2 = head_inp @ head_q_proj_weight.T
print(f"{head_query_out_2.shape=}")

q_proj_weight_split.shape=torch.Size([64, 128, 8192])
head_q_proj_weight.shape=torch.Size([128, 8192])
head_query_out_2.shape=torch.Size([1, 43, 128])


In [245]:
print(torch.allclose(head_query_out, head_query_out_2, atol=1e-1))
(head_query_out_2 - head_query_out)

False


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:3',
       dtype=torch.bfloat16, grad_fn=<SubBackward0>)

In [246]:
head_q_proj_weight.shape

torch.Size([128, 8192])

In [247]:
head_inp.shape

torch.Size([1, 43, 8192])

In [251]:
with torch.no_grad():
    head_q_proj = head_q_proj_weight.to(torch.float32)
    proj_matrix = head_q_proj.T @ torch.linalg.pinv(head_q_proj @ head_q_proj.T) @ head_q_proj
    head_query_read = head_inp @ proj_matrix.to(mt.dtype).T

ll_pred = logit_lens(
    mt = mt,
    h = head_query_read[:, -1].squeeze(),
    k=15
)
print(f'll_pred={[str(pred) for pred in ll_pred]}')

patch_pred = patchscope(
    mt=mt,
    h=head_query_read[:, -1].squeeze(),
    context=context_prof,
    placeholder="placeholder",
    patch_layers=patch_layers,
    k=15,
)
print(f'patch_pred={[str(pred) for pred in patch_pred]}')

proj_matrix.shape

ll_pred=['" Gow"[93571] (p=0.037, logit=9.562)', '" stalk"[55972] (p=0.003, logit=7.125)', '" Bag"[20287] (p=0.003, logit=6.906)', '"groupon"[71014] (p=0.002, logit=6.844)', '"alaxy"[109956] (p=0.002, logit=6.812)', '" bip"[29978] (p=0.002, logit=6.781)', '" ong"[127721] (p=0.002, logit=6.656)', '"ंर"[123418] (p=0.002, logit=6.531)', '" bag"[9145] (p=0.002, logit=6.531)', '" fre"[3541] (p=0.002, logit=6.469)', '" Shepherd"[59646] (p=0.002, logit=6.375)', '" Collector"[59648] (p=0.002, logit=6.375)', '"/oauth"[61803] (p=0.001, logit=6.312)', '" Trip"[27852] (p=0.001, logit=6.312)', '"kili"[118462] (p=0.001, logit=6.312)']
patch_pred=['" invalid"[8482] (p=0.070, logit=10.875)', '" unknown"[9987] (p=0.058, logit=10.688)', '"??"[9602] (p=0.055, logit=10.625)', '"?\n"[18072] (p=0.048, logit=10.500)', '"???"[52417] (p=0.045, logit=10.438)', '"?\n\n"[24688] (p=0.033, logit=10.125)', '" error"[1493] (p=0.031, logit=10.062)', '"?"[949] (p=0.028, logit=9.938)', '" not"[539] (p=0.021, logit=9.688

torch.Size([8192, 8192])

In [252]:
head_inp.shape, proj_matrix.shape

(torch.Size([1, 43, 8192]), torch.Size([8192, 8192]))